In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('test').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/20 09:22:58 WARN Utils: Your hostname, Harishankars-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.114 instead (on interface en0)
26/01/20 09:22:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/20 09:22:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# 1) Calculate the sum of a column in a DataFrame
data = [
    ("John Doe", "john@example.com", 50000.0),
    ("Jane Smith", "jane@example.com", 60000.0),
    ("Bob Johnson", "bob@example.com", 55000.0)
]

schema = "Name string, email string, salary double"
df = spark.createDataFrame(data, schema)

In [12]:
from pyspark.sql.functions import *
df.agg(sum('salary')).show()

+-----------+
|sum(salary)|
+-----------+
|   165000.0|
+-----------+



In [18]:
# 2) Get all marks for each individual in a list
data=[(1,'Watson',34),(1,'Watson',40),(1,'Watson',34),
      (2,'Alex',45),(2,'Alex',50)]

schema="ID int, Name string, Marks int"
df=spark.createDataFrame(data,schema)

In [17]:
df.groupBy('name').agg(sum('marks').alias('total_marks')).show()

+------+-----------+
|  name|total_marks|
+------+-----------+
|Watson|        108|
|  Alex|         95|
+------+-----------+



In [19]:
# 3) Identify rows with non-numeric quantity
from pyspark.sql.types import *

schema = StructType([
  StructField("ProductCode", StringType(), True),
  StructField("Quantity", StringType(), True),
  StructField("UnitPrice", StringType(), True),
  StructField("CustomerID", StringType(), True)
])

data = [
  ("Q001", "5", "20.0", "C001"),
  ("Q002", "3", "15.5", "C002"),
  ("Q003", "10", "5.99", "C003"),
  ("Q004", "2", "50.0", "C001"),
  ("Q005", "nein", "12.75", "C002")
]

df = spark.createDataFrame(data, schema)

In [21]:
df.filter(~col('quantity').rlike("^\\d+$")).show()

+-----------+--------+---------+----------+
|ProductCode|Quantity|UnitPrice|CustomerID|
+-----------+--------+---------+----------+
|       Q005|    nein|    12.75|      C002|
+-----------+--------+---------+----------+



In [28]:
#4) Split comma-separated hobbies into multiple rows
data=[('Paris','Polo,Tennis'),('Matt','Golf,Hockey'),('Sam',None)]
schema="Name string, Hobbies string"
df=spark.createDataFrame(data,schema)

In [30]:
df2 = df.withColumn("Hobbies", split(col("Hobbies"), ",")) \
        .withColumn("Hobbies", explode(col("Hobbies")))
df2.show()

+-----+-------+
| Name|Hobbies|
+-----+-------+
|Paris|   Polo|
|Paris| Tennis|
| Matt|   Golf|
| Matt| Hockey|
+-----+-------+



In [38]:
# 5) Get year, start_week_date, end_week_date per week
from pyspark.sql.types import *

data=[(2025,1,'2025-01-01'),(2025,1,'2025-01-02'),(2025,1,'2025-01-03'),
      (2025,1,'2025-01-04'),(2025,1,'2025-01-05'),(2025,1,'2025-01-06'),
      (2025,1,'2025-01-07'),(2025,2,'2025-01-08'),(2025,2,'2025-01-09'),
      (2025,2,'2025-01-10'),(2025,2,'2025-01-11'),(2025,2,'2025-01-12'),
      (2025,2,'2025-01-13'),(2025,2,'2025-01-14')]

schema=StructType([
    StructField('year',IntegerType(),True),
    StructField('week_num',IntegerType(),True),
    StructField('dates',StringType(),True)
])

df=spark.createDataFrame(data,schema)

In [37]:
df.groupBy('year', 'week_num').agg(
    min(col('dates')).alias('min_date'),
    max(col('dates')).alias('max_date')
).show()

+----+--------+----------+----------+
|year|week_num|  min_date|  max_date|
+----+--------+----------+----------+
|2025|       1|2025-01-01|2025-01-07|
|2025|       2|2025-01-08|2025-01-14|
+----+--------+----------+----------+



In [39]:
# 6) Compute shipped and delivered percentages
from pyspark.sql.types import *

data = [
 (1, "Shipped", "2025-04-01"),
 (1, "Shipped", "2025-04-02"),
 (2, "Delivered", "2025-04-01"),
 (2, "Delivered", "2025-04-02"),
 (2, "Shipped", "2025-04-03"),
 (3, "Shipped", "2025-04-01"),
 (3, "Delivered", "2025-04-02"),
 (3, "Delivered", "2025-04-03")
]

schema = StructType([
 StructField("order_id", IntegerType(), False),
 StructField("order_status", StringType(), True),
 StructField("order_date", StringType(), True)
])

orders_df = spark.createDataFrame(data, schema)
orders_df.show()

+--------+------------+----------+
|order_id|order_status|order_date|
+--------+------------+----------+
|       1|     Shipped|2025-04-01|
|       1|     Shipped|2025-04-02|
|       2|   Delivered|2025-04-01|
|       2|   Delivered|2025-04-02|
|       2|     Shipped|2025-04-03|
|       3|     Shipped|2025-04-01|
|       3|   Delivered|2025-04-02|
|       3|   Delivered|2025-04-03|
+--------+------------+----------+



In [47]:
from pyspark.sql.functions import col, count, when

# total
total_order_count = (
    orders_df.groupBy("order_id")
             .agg(count("*").alias("total_orders"))
)

# shipped
shipped_count = (
    orders_df.filter(col("order_status") == "Shipped")
             .groupBy("order_id")
             .agg(count("*").alias("shipped_orders"))
)

# delivered
delivered_count = (
    orders_df.filter(col("order_status") == "Delivered")
             .groupBy("order_id")
             .agg(count("*").alias("delivered_orders"))
)

# join and fill missing with 0
order_status_count = (
    total_order_count
        .join(shipped_count, "order_id", "left")
        .join(delivered_count, "order_id", "left")
        .fillna(0)
)

# percentages
df_final = (
    order_status_count
        .withColumn("shipped_percentage", (col("shipped_orders") / col("total_orders")) * 100)
        .withColumn("delivered_percentage", (col("delivered_orders") / col("total_orders")) * 100)
        .select("order_id", "shipped_percentage", "delivered_percentage")
)

df_final.show()

+--------+------------------+--------------------+
|order_id|shipped_percentage|delivered_percentage|
+--------+------------------+--------------------+
|       1|             100.0|                 0.0|
|       2| 33.33333333333333|   66.66666666666666|
|       3| 33.33333333333333|   66.66666666666666|
+--------+------------------+--------------------+



26/01/07 11:34:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1045899 ms exceeds timeout 120000 ms
26/01/07 11:34:36 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/07 11:51:30 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

In [3]:
order_df = spark.read.csv(r'/Users/harishankargiri/Desktop/Data Engineering/PySpark/PractiseByGPT/orders.csv', header = True, inferSchema=True)
order_df.head()

Row(order_id=1001, customer_id=6, order_date=datetime.date(2023, 10, 5), status='Cancelled')

In [14]:
order_df.count()

20

In [10]:
order_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- status: string (nullable = true)



In [15]:
order_df.cache()

DataFrame[order_id: int, customer_id: int, order_date: date, status: string]

In [16]:
p_df = spark.read.csv(r'/Users/harishankargiri/Desktop/Data Engineering/PySpark/PractiseByGPT/products.csv', header = True, inferSchema=True)


In [23]:
p_df.groupBy('category').count().show()

+-----------+-----+
|   category|count|
+-----------+-----+
|Electronics|    2|
|   Clothing|    6|
|      Books|    6|
|  Furniture|    6|
+-----------+-----+



In [25]:
p_df.select('category').distinct().show()

+-----------+
|   category|
+-----------+
|Electronics|
|   Clothing|
|      Books|
|  Furniture|
+-----------+



### 19th Jan 2026

In [2]:
cust_df = spark.read.csv(r'/Users/harishankargiri/Desktop/Data Engineering/PySpark/PractiseByGPT/customers.csv', header=True, inferSchema=True)
item_df = spark.read.csv(r'/Users/harishankargiri/Desktop/Data Engineering/PySpark/PractiseByGPT/items.csv', header=True, inferSchema=True)
order_df = spark.read.csv(r'/Users/harishankargiri/Desktop/Data Engineering/PySpark/PractiseByGPT/orders.csv', header=True, inferSchema=True)
payment_df = spark.read.csv(r'/Users/harishankargiri/Desktop/Data Engineering/PySpark/PractiseByGPT/payments.csv', header=True, inferSchema=True)
product_df = spark.read.csv(r'/Users/harishankargiri/Desktop/Data Engineering/PySpark/PractiseByGPT/products.csv', header=True, inferSchema=True)

Question 1: Customer Order Funnel (Real-World Analytics)

For each city, calculate:

Total number of customers

Number of customers who have placed at least one order

Number of customers who have placed at least one completed order (status = 'Delivered')

👉 Output should be one row per city.

In [46]:
cust_order_df = cust_df.join(order_df, on='customer_id', how='left')

from pyspark.sql import functions as F
cust_order_df.groupBy('city').agg(
    F.countDistinct('customer_id').alias('no_of_cust'),
    F.count('order_id').alias('active_cust'),
    F.sum(F.when(F.col('status') == 'Delivered', 1).otherwise(0)).alias('delivered_cust')
).show()

+---------+----------+-----------+--------------+
|     city|no_of_cust|active_cust|delivered_cust|
+---------+----------+-----------+--------------+
|Bangalore|         6|          6|             3|
|  Chennai|         3|          4|             1|
|   Mumbai|         3|          3|             1|
|  Kolkata|         5|          7|             2|
|    Delhi|         3|          0|             0|
+---------+----------+-----------+--------------+



🔹 Question 2: Revenue Leakage Check (Very Common in Industry)

Find all orders where:

The sum of payment amounts does NOT match

The expected order value (sum of quantity × product price)

Return:
order_id,
expected_order_amount,
paid_amount,
difference

👉 This tests joins + aggregation + business validation.

In [70]:
order_payment_item_df = order_df.join(payment_df, on='order_id', how='left').join(item_df, on='order_id', how='inner')
ans_df = order_payment_item_df.groupBy('order_id')\
    .agg(
        F.sum(F.col('quantity') * F.col('price')).alias('expected_amount'),
        F.max(F.col('amount')).alias('paid_amount')
    )\
    .withColumn('Difference', F.col('paid_amount') - F.col('expected_amount'))\
    .show()

+--------+---------------+-----------+----------+
|order_id|expected_amount|paid_amount|Difference|
+--------+---------------+-----------+----------+
|    1005|          67000|      74728|      7728|
|    1016|          16400|      77109|     60709|
|    1019|          37900|      10752|    -27148|
|    1008|          90000|      74756|    -15244|
|    1010|          20000|      79036|     59036|
|    1002|          38000|      81505|     43505|
|    1017|         106000|      69613|    -36387|
|    1015|         111000|      65080|    -45920|
|    1001|          36500|      25658|    -10842|
|    1020|          23000|      73679|     50679|
|    1006|          17500|      48462|     30962|
|    1007|          46000|      40836|     -5164|
|    1003|          34000|      84427|     50427|
|    1014|          50400|      94866|     44466|
|    1018|          42000|      50575|      8575|
|    1004|          14400|      43819|     29419|
|    1011|          66800|      54744|    -12056|


🔹 Question 3: Customer Purchase Behavior (Window Function)

For each customer, find:

Their first order date

Their latest order date

Number of days baetween first and latest order

Total number of orders placed

👉 Use window functions, not groupBy alone.

In [97]:
cust_order_df.groupBy('customer_id').agg(
    F.count_distinct('order_id').alias('no_of_order'),
    F.min('order_date').alias('first_order'),
    F.max('order_date').alias('latest_order')
).withColumn(
    'days_diff',
    F.datediff(F.col('latest_order'), F.col('first_order'))
).dropna().show()

+-----------+-----------+-----------+------------+---------+
|customer_id|no_of_order|first_order|latest_order|days_diff|
+-----------+-----------+-----------+------------+---------+
|         12|          3| 2023-02-28|  2023-09-12|      196|
|         13|          1| 2023-03-29|  2023-03-29|        0|
|          6|          2| 2023-08-25|  2023-10-05|       41|
|         16|          1| 2023-08-01|  2023-08-01|        0|
|          5|          1| 2023-09-15|  2023-09-15|        0|
|         15|          2| 2023-06-27|  2023-12-31|      187|
|          9|          1| 2023-04-05|  2023-04-05|        0|
|          8|          2| 2023-02-20|  2023-04-28|       67|
|          7|          1| 2023-03-28|  2023-03-28|        0|
|         10|          1| 2023-04-17|  2023-04-17|        0|
|         11|          2| 2023-03-12|  2023-11-09|      242|
|          2|          1| 2023-12-08|  2023-12-08|        0|
|         18|          2| 2023-04-11|  2023-10-17|      189|
+-----------+-----------

🔹 Question 4: Top Product per Category (Interview Favorite)

For each product category, find the top-selling product based on:

Total quantity sold

If two products tie, choose the one with:

Higher total revenue (quantity × price)

👉 Output one product per category.

In [102]:
p = product_df.alias("p")
i = item_df.alias("i")
o = order_df.alias("o")

product_order_item_df = (
    p.join(i, on="product_id", how="inner")
     .join(o, on="order_id", how="inner")
)

In [106]:
df = product_order_item_df.select(
    F.col("p.product_id"),
    F.col("p.name"),
    F.col("p.category"),
    F.col("p.price"),
    F.col("i.quantity")
)

df.show()

agg_df = df.groupBy('category', 'product_id', 'name') \
.agg(
    F.sum('quantity').alias('quantity_sold'),
    F.sum(F.col('quantity') * F.col('price')).alias('total_revenue')
)

+----------+----------+-----------+-----+--------+
|product_id|      name|   category|price|quantity|
+----------+----------+-----------+-----+--------+
|       117|Product_17|   Clothing| 6000|       4|
|       115|Product_15|      Books| 2500|       5|
|       114|Product_14|  Furniture|16000|       2|
|       117|Product_17|   Clothing| 6000|       1|
|       120|Product_20|      Books| 2000|       2|
|       117|Product_17|   Clothing| 6000|       5|
|       103| Product_3|   Clothing| 3800|       2|
|       110|Product_10|   Clothing| 6800|       1|
|       104| Product_4|      Books| 2200|       5|
|       106| Product_6|Electronics|14000|       4|
|       115|Product_15|      Books| 2500|       4|
|       115|Product_15|      Books| 2500|       3|
|       120|Product_20|      Books| 2000|       2|
|       118|Product_18|  Furniture|14000|       3|
|       111|Product_11|  Furniture|15000|       5|
|       119|Product_19|      Books| 3000|       5|
|       103| Product_3|   Cloth

In [111]:
from pyspark.sql.window import Window

window_spec = Window.partitionBy("category") \
    .orderBy(
        F.desc("quantity_sold"),
        F.desc("total_revenue")
    )

result_df = agg_df.withColumn(
    "rank",
    F.row_number().over(window_spec)
).filter(F.col('rank') == 1).show()

+-----------+----------+----------+-------------+-------------+----+
|   category|product_id|      name|quantity_sold|total_revenue|rank|
+-----------+----------+----------+-------------+-------------+----+
|      Books|       115|Product_15|           16|        40000|   1|
|   Clothing|       105| Product_5|           13|        41600|   1|
|Electronics|       108| Product_8|            5|        45000|   1|
|  Furniture|       102| Product_2|           13|       234000|   1|
+-----------+----------+----------+-------------+-------------+----+



🔹 Question 5: Payment Type Distribution (Business Insight)

For each payment_type, calculate:

Total number of orders paid using that payment type

Total revenue generated

Percentage contribution to overall revenue

👉 Percentage should be calculated using PySpark (not hardcoded).

In [31]:
from pyspark.sql import functions as F
agg_df = payment_df.groupBy('payment_type').agg(
    F.count(F.col('order_id')).alias('total_orders'),
    F.sum(F.col('amount')).alias('total_revenue')
)

total_value = payment_df.agg(F.sum('amount')).collect()[0][0]

result = agg_df.withColumn(
    'percentage',
    F.col('total_revenue') * 100 / F.lit(total_value)
)

result.show()


+------------+------------+-------------+------------------+
|payment_type|total_orders|total_revenue|        percentage|
+------------+------------+-------------+------------------+
|  NetBanking|           5|       271256|23.711188811188812|
|        Card|           5|       274568|  24.0006993006993|
|      Wallet|           4|       215915| 18.87368881118881|
|         UPI|           6|       382261| 33.41442307692308|
+------------+------------+-------------+------------------+



### 20th Jan 2026

Find customers who:

Signed up more than 90 days ago

Have placed at least one order

Have not placed any order in the last 60 days

Return:
customer_id
name
city
last_order_date
total_orders

👉 Tests joins, date logic, and anti-patterns.

In [10]:
from pyspark.sql import functions as F

cust_order_df = cust_df.join(
    order_df,
    on='customer_id',
    how='right'
)

result_df = (
    cust_order_df
    .groupBy('customer_id')
    .agg(
        F.min('name').alias('name'),
        F.min('city').alias('city'),
        F.min('signup_date').alias('signup_date'),
        F.max('order_date').alias('latest_date'),
        F.count('order_id').alias('total_orders')
    )
    .filter(
        F.datediff(F.current_date(), F.col('signup_date')) > 90
    )
)

result_df.show()

+-----------+-----------+---------+-----------+-----------+------------+
|customer_id|       name|     city|signup_date|latest_date|total_orders|
+-----------+-----------+---------+-----------+-----------+------------+
|          2| Customer_2|   Mumbai| 2023-01-12| 2023-12-08|           1|
|          5| Customer_5|  Kolkata| 2023-02-01| 2023-09-15|           1|
|          6| Customer_6|Bangalore| 2023-08-02| 2023-10-05|           2|
|          7| Customer_7|Bangalore| 2023-07-30| 2023-03-28|           1|
|          8| Customer_8|  Chennai| 2023-08-21| 2023-04-28|           2|
|          9| Customer_9|   Mumbai| 2023-08-10| 2023-04-05|           1|
|         10|Customer_10|Bangalore| 2023-08-22| 2023-04-17|           1|
|         11|Customer_11|  Chennai| 2023-01-26| 2023-11-09|           2|
|         12|Customer_12|  Kolkata| 2023-10-21| 2023-09-12|           3|
|         13|Customer_13|  Kolkata| 2023-03-25| 2023-03-29|           1|
|         15|Customer_15|  Kolkata| 2023-12-18| 202

🔹 Question 2: Order Completion Ratio (Business KPI)

For each city, calculate:

Total number of orders

Number of completed orders (status = 'COMPLETED')

Order completion ratio = completed_orders / total_orders

Sort cities by lowest completion ratio first.

In [17]:
cust_order_df.groupBy('city').agg(
    F.count('order_id').alias('total_orders'),
    F.count(
        F.when(F.col('status') == 'Delivered', 1)
    ).alias('delivered_orders'),
).withColumn(
    'order_completion_ration',
    F.round(F.col('delivered_orders') * 100 / F.col('total_orders'), 2)
).show()

+---------+------------+----------------+-----------------------+
|     city|total_orders|delivered_orders|order_completion_ration|
+---------+------------+----------------+-----------------------+
|Bangalore|           6|               3|                   50.0|
|  Chennai|           4|               1|                   25.0|
|   Mumbai|           3|               1|                  33.33|
|  Kolkata|           7|               2|                  28.57|
+---------+------------+----------------+-----------------------+



🔹 Question 3: Product Demand Consistency (Window Function)

For each product, calculate:

Total quantity sold

Average quantity sold per order

Rank products within each category by average quantity per order

👉 Use window functions, not only groupBy.

🔹 Question 4: Multiple Payment Detection (Data Quality Check)

Find orders where:

More than one payment record exists for the same order

Return:

order_id

number_of_payments

total_payment_amount

👉 Very common fraud / data-quality interview question.

🔹 Question 5: Customer Lifetime Value (CLV – Entry Level Version)

For each customer, calculate:

Total revenue generated

Number of distinct orders

Average revenue per order

Only include customers who:

Have placed at least 2 orders

Sort by total revenue descending.